In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

###Silver Data Transformation

In [0]:
df = spark.read.format("delta")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@netflixprojectdlparas.dfs.core.windows.net/netflix_titles")

In [0]:
df = df.fillna({"duration_minutes": 0, "duration_seasons": 1})
df = df.withColumn("duration_minutes", col("duration_minutes").cast(IntegerType()))\
    .withColumn("duration_seasons", col("duration_seasons").cast(IntegerType()))

In [0]:
df = df.withColumn("short_title",split(col("title"), ":").getItem(0) )\
    .withColumn("rating",split(col("rating"),"-").getItem(0) )

In [0]:
df = df.withColumn("type_flag",when(col("type")=="Movie",1)\
        .when(col("type")=="TV Show",2).otherwise(0) )

In [0]:
from pyspark.sql.window import Window

In [0]:
df = df.withColumn("duration_ranking",dense_rank().over(Window.orderBy(col("duration_minutes").desc() ) ) )

In [0]:
df_visualisation = df.groupBy("type").agg(count("*").alias("total_count") ).display()

Databricks visualization. Run in Databricks to view.

In [0]:
df.write.format("delta")\
    .mode("overwrite")\
    .option("path","abfss://silver@netflixprojectdlparas.dfs.core.windows.net/netflix_titles")\
    .save()